In [1]:
using TSSM

 *** OPENMP n_threads =           4


In [2]:
nx = 1024
xmin = -16
xmax = +16
meth = Schroedinger1D(nx, xmin, xmax, cubic_coupling=-1)

TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x000000000365f220)

In [3]:
# exact solution
function soliton(x, t)
    a = 2.0
    b = 1.0
    c = 0.0    
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [4]:
V(x,t) = 0.5*abs(soliton(x,t)).^2
method = Schroedinger1D(nx, xmin, xmax, potential_t=V, cubic_coupling=-0.5)

TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x0000000003bbff70)

In [5]:
# cubic_coupling=-1 has to be multiplied by -1im 
# because of the factor 1im at the lefthand side of the Schrödinger equation
F(t, x, u) = 0.5*1im*conj(u)*u^2 + 0.5*1im*abs(soliton(x,t)).^2*u
function gen_rhs!(rhs::WfSchroedinger1D, psi::WfSchroedinger1D)
    to_real_space!(rhs)
    to_real_space!(psi)
    t = get_time(psi)
    u = get_data(psi, true)
    u1 = get_data(rhs, true)
    x = get_nodes(psi.m)
    for j=1:length(u)
        u1[j] = F(t, x[j], u[j])
    end
end

gen_rhs! (generic function with 1 method)

In [6]:
t0 = 0.0
tend = 1.0
psi = wave_function(meth)
psi_ref = wave_function(meth)

TSSM.WfSchroedinger1D{Float64}(Ptr{Void} @0x0000000003ba5ef0,TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x000000000365f220))

In [7]:
include("time_propagators.jl")

step! (generic function with 4 methods)

# One-step methods

## Explicit Runge-Kutta
### Classical Runge-Kutta scheme of order 4

In [8]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
method = ExponentialRungeKutta(:rk4)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01         NaN
  2   1.250e-01         NaN    NaN
  3   6.250e-02         NaN    NaN
  4   3.125e-02         NaN    NaN
  5   1.563e-02         NaN    NaN
  6   7.813e-03         NaN    NaN
  7   3.906e-03         NaN    NaN
  8   1.953e-03         NaN    NaN
  9   9.766e-04         NaN    NaN
 10   4.883e-04   4.933e-13    NaN


## Splitting Method
### 5-step composition of order 4 (Suzuki)

In [9]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
method = SplittingMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.184e-02
  2   1.250e-01   4.867e-03   2.71
  3   6.250e-02   3.391e-04   3.84
  4   3.125e-02   8.288e-06   5.35
  5   1.563e-02   3.653e-07   4.50
  6   7.813e-03   2.266e-08   4.01
  7   3.906e-03   1.414e-09   4.00
  8   1.953e-03   8.834e-11   4.00
  9   9.766e-04   5.289e-12   4.06
 10   4.883e-04   9.566e-13   2.47


### 3-step composition of order 4 (Yoshida)

In [10]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
a, b = get_coeffs_composition(g)
method = SplittingMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.696e-01
  2   1.250e-01   6.568e-02   2.49
  3   6.250e-02   5.800e-03   3.50
  4   3.125e-02   3.752e-04   3.95
  5   1.563e-02   2.397e-05   3.97
  6   7.813e-03   1.507e-06   3.99
  7   3.906e-03   9.435e-08   4.00
  8   1.953e-03   5.899e-09   4.00
  9   9.766e-04   3.688e-10   4.00
 10   4.883e-04   2.275e-11   4.02


## Composition Method, B-part: implizit midpoint rule
### 5-step composition of order 4 (Suzuki), 3 fixed-point iterations

In [11]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
method = CompositionMethod(g,2)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.357e-01
  2   1.250e-01   5.777e-02   2.54
  3   6.250e-02   7.520e-03   2.94
  4   3.125e-02   9.324e-04   3.01
  5   1.563e-02   1.155e-04   3.01
  6   7.813e-03   1.436e-05   3.01
  7   3.906e-03   1.790e-06   3.00
  8   1.953e-03   2.234e-07   3.00
  9   9.766e-04   2.790e-08   3.00
 10   4.883e-04   3.486e-09   3.00


### 4 fixed-point iterations

In [12]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
method = CompositionMethod(g,3)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   4.153e-02
  2   1.250e-01   5.614e-03   2.89
  3   6.250e-02   4.207e-04   3.74
  4   3.125e-02   8.396e-06   5.65
  5   1.563e-02   2.256e-07   5.22
  6   7.813e-03   1.433e-08   3.98
  7   3.906e-03   9.139e-10   3.97
  8   1.953e-03   5.780e-11   3.98
  9   9.766e-04   3.741e-12   3.95
 10   4.883e-04   9.907e-13   1.92


### 3-step composition of order 4 (Yoshida), 3 fixed-point iterations

In [13]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
method = CompositionMethod(g,3)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   9.449e-01
  2   1.250e-01   2.432e-01   1.96
  3   6.250e-02   1.409e-02   4.11
  4   3.125e-02   4.193e-04   5.07
  5   1.563e-02   1.015e-05   5.37
  6   7.813e-03   3.930e-07   4.69
  7   3.906e-03   3.126e-08   3.65
  8   1.953e-03   2.312e-09   3.76
  9   9.766e-04   1.569e-10   3.88
 10   4.883e-04   1.033e-11   3.93


## Exponential Runge-Kutta 
### Krogstad scheme of order 4

In [14]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
method = ExponentialRungeKutta(:krogstad)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.870e-02
  2   1.250e-01   4.560e-03   3.09
  3   6.250e-02   3.267e-04   3.80
  4   3.125e-02   2.040e-05   4.00
  5   1.563e-02   1.245e-06   4.03
  6   7.813e-03   7.638e-08   4.03
  7   3.906e-03   4.721e-09   4.02
  8   1.953e-03   2.933e-10   4.01
  9   9.766e-04   1.827e-11   4.00
 10   4.883e-04   1.158e-12   3.98


### Strehmel-Weiner scheme of order 4

In [15]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
method = ExponentialRungeKutta(:strehmel_weiner)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   5.246e-02
  2   1.250e-01   5.453e-03   3.27
  3   6.250e-02   3.831e-04   3.83
  4   3.125e-02   2.401e-05   4.00
  5   1.563e-02   1.475e-06   4.03
  6   7.813e-03   9.086e-08   4.02
  7   3.906e-03   5.631e-09   4.01
  8   1.953e-03   3.503e-10   4.01
  9   9.766e-04   2.184e-11   4.00
 10   4.883e-04   1.376e-12   3.99


# Multistep methods

In [16]:
include("time_propagators.jl")

step! (generic function with 4 methods)

In [17]:
type ExactSolution <: TimePropagationMethod
end
function step!(m::ExactSolution, psi::WaveFunction, 
         t0::Real, dt::Real, steps::Int, step::Int)
    set!(psi, soliton, t0+(step+1)*dt)
end    

step! (generic function with 5 methods)

## Exponential multistep
### exact evaluation of the integral; 4 steps; exact starting values; no correction step

In [18]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
method = ExponentialMultistep(4, iters=0, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   4.888e-01
  2   1.250e-01   6.979e-02   2.81
  3   6.250e-02   4.556e-03   3.94
  4   3.125e-02   2.948e-04   3.95
  5   1.563e-02   1.849e-05   3.99
  6   7.813e-03   1.153e-06   4.00
  7   3.906e-03   7.193e-08   4.00
  8   1.953e-03   4.490e-09   4.00
  9   9.766e-04   2.804e-10   4.00
 10   4.883e-04   1.748e-11   4.00


### exact evaluation of the integral; 3 steps; exact starting values; one correction step

In [19]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
method = ExponentialMultistep(3, iters=1, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   8.324e-02
  2   1.250e-01   1.877e-02   2.15
  3   6.250e-02   2.168e-03   3.11
  4   3.125e-02   1.585e-04   3.77
  5   1.563e-02   1.013e-05   3.97
  6   7.813e-03   6.304e-07   4.01
  7   3.906e-03   3.915e-08   4.01
  8   1.953e-03   2.436e-09   4.01
  9   9.766e-04   1.519e-10   4.00
 10   4.883e-04   9.442e-12   4.01


### Gauss quadrature of order 4 for the  integral; 3 steps; exact starting values; one correction step

In [20]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
gauss4 = QuadratureRule([(3-sqrt(3))/6, (3+sqrt(3))/6],[0.5,0.5])
method = ExponentialMultistep(3, iters=1, quadrature=gauss4, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   1.414e-01
  2   1.250e-01   2.588e-02   2.45
  3   6.250e-02   2.443e-03   3.41
  4   3.125e-02   1.661e-04   3.88
  5   1.563e-02   1.058e-05   3.97
  6   7.813e-03   6.585e-07   4.01
  7   3.906e-03   4.091e-08   4.01
  8   1.953e-03   2.547e-09   4.01
  9   9.766e-04   1.587e-10   4.00
 10   4.883e-04   9.851e-12   4.01


### Exponential multistep version 2; 3 steps; exact starting values; one correction step

In [21]:
set!(psi, soliton, t0)
set!(psi_ref, soliton, tend)
method = ExponentialMultistep(3, iters=1, version=2, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.885e-01
  2   1.250e-01   9.816e-02   1.98
  3   6.250e-02   1.049e-02   3.23
  4   3.125e-02   1.964e-03   2.42
  5   1.563e-02   2.065e-04   3.25
  6   7.813e-03   1.463e-05   3.82
  7   3.906e-03   9.240e-07   3.98
  8   1.953e-03   5.719e-08   4.01
  9   9.766e-04   3.543e-09   4.01
 10   4.883e-04   2.202e-10   4.01


### Exponential multistep version 2; 3 steps; starting values by Suzuki composition method; one correction step

In [22]:
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
#starting_method = SplittingMethod(a,b)
starting_method = CompositionMethod(g,3)
method = ExponentialMultistep(3, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err      p
-----------------------------------
  1   2.500e-01   3.755e-01
  2   1.250e-01   9.767e-02   1.94
  3   6.250e-02   1.048e-02   3.22
  4   3.125e-02   1.964e-03   2.42
  5   1.563e-02   2.065e-04   3.25
  6   7.813e-03   1.463e-05   3.82
  7   3.906e-03   9.240e-07   3.98
  8   1.953e-03   5.719e-08   4.01
  9   9.766e-04   3.543e-09   4.01
 10   4.883e-04   2.202e-10   4.01
